Using kernel `conda_pytorch_latest_p36`

In [1]:
# !pip install transformers
# !pip install datasets

In [6]:
import sys
sys.path.append('../../../')

In [7]:
from pathlib import Path
import os
import random
import pickle

In [8]:
import pandas as pd
import numpy as np
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
from sklearn.metrics import classification_report, precision_recall_fscore_support, precision_score, f1_score, recall_score
from sklearn.preprocessing import MultiLabelBinarizer
import torch

In [9]:
from deep.constants import *

In [20]:
train = pd.read_csv(LATEST_DATA_PATH / 'data_v0.4.2_train.csv')
val = pd.read_csv(LATEST_DATA_PATH / 'data_v0.4.2_val.csv')
test = pd.read_csv(LATEST_DATA_PATH / 'data_v0.4.2_test.csv')

def process(df):
    df = df.copy()
    df['sectors'] = df['sectors'].apply(eval)
#     df['dimension_ids'] = df['dimension_ids'].apply(lambda x: torch.tensor(x, dtype=torch.float))
    
    mlb = MultiLabelBinarizer()
    labels = mlb.fit_transform(list(df['sectors']))
    df['labels'] = list(labels)
    
    df = df[['excerpt', 'labels']]
    df = df.rename(columns={'excerpt': 'texts'})
        
    return df

train_df = process(train)
val_df = process(val)
test_df = process(test)

In [21]:
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

In [22]:
a = pd.read_pickle('/Users/stefano/Downloads/test_predictions.pickle')
b = sigmoid(a)

In [23]:
a

tensor([[-0.5335,  0.0302,  0.4279,  ..., -0.1528,  0.0892, -0.2532],
        [-0.4693,  0.2286,  0.4292,  ..., -0.2542,  0.2493, -0.0855],
        [-0.6634,  0.1816,  0.1597,  ..., -0.0674,  0.3895, -0.0136],
        ...,
        [-0.5865,  0.1427,  0.5291,  ..., -0.2105,  0.1357, -0.0486],
        [-0.6558,  0.2421,  0.2595,  ..., -0.0424,  0.1129, -0.1527],
        [-0.5395,  0.3054,  0.2341,  ..., -0.0561,  0.1751, -0.1357]])

In [24]:
b

tensor([[0.3697, 0.5075, 0.6054,  ..., 0.4619, 0.5223, 0.4370],
        [0.3848, 0.5569, 0.6057,  ..., 0.4368, 0.5620, 0.4786],
        [0.3400, 0.5453, 0.5399,  ..., 0.4831, 0.5962, 0.4966],
        ...,
        [0.3574, 0.5356, 0.6293,  ..., 0.4476, 0.5339, 0.4879],
        [0.3417, 0.5602, 0.5645,  ..., 0.4894, 0.5282, 0.4619],
        [0.3683, 0.5758, 0.5583,  ..., 0.4860, 0.5437, 0.4661]])

In [25]:
b.shape

torch.Size([11192, 11])

In [26]:
test_df.shape

(11192, 2)

In [35]:
thresh = 0.55
index=4


preds = (b > thresh).long()
labels = torch.tensor(test_df.labels)

preds = preds[:,index]
labels = labels[:, index]

prec = precision_score(labels, preds)
rec = recall_score(labels, preds)
precision_recall_fscore_support(labels, preds, average="binary")

(0.39364035087719296, 0.11447704081632654, 0.1773715415019763, None)